# Project: Kaggle Challenge Movie Rating Prediction 
In this project, we write a K-Nearest Neighbor algorithm to predict IMDB rating of movies. We use a dataset Kaggle that contains 28 variables for 5043 movies, spanning across 100 years in 66 countries. 
We write our K-Nearest Neighbor algorithm from scratch without using built-in libraries.
We’ll implement the three steps of the K-Nearest Neighbor Algorithm:
- Normalize the data
- Find the k nearest neighbors
- Classify the new point based on those neighbors

In [19]:
import pandas as pd

## Part 1: Load the data

In [22]:
# https://www.kaggle.com/carolzhangdc/predict-imdb-score-with-data-mining-algorithms
# The dataset is from Kaggle website. It contains 28 variables for 5043 movies, spanning across 100 years in 66 countries. 
df = pd.read_csv('08 ML_KNearestNeighbors_KaggleChallengeMovieRatingPredictionDataset.csv')
# Make a subset of the dataframe with relevant features to use for our K-Nearest Neighbor 
movie_dataset = df[['movie_title', 'duration', 'actor_1_facebook_likes', 'num_critic_for_reviews','budget','gross','title_year','movie_facebook_likes']]
movie_dataset.head(3)
# Make a subset of the dataframe with IMDB score 
movie_labels = df[['movie_title','imdb_score']]
movie_labels.head(3)
# Adding a new column diviging movies into bad movie (IMDB rating < 7) and good (IMDB rating >= 7)
movie_labels['l'] = movie_labels.apply(lambda row: 1 if row['imdb_score'] >= 7 else 0, axis=1)
movie_labels.head(3)

,movie_title,imdb_score,l
0,Avatar,7.9,1
1,Pirates of the Caribbean: At World's End,7.1,1
2,Spectre,6.8,0


## Part 2: Normalize the data

## Part 3: Distance function 

In [24]:
# Distance Between Points using Euclidean Distance
def distance(movie1, movie2):
  length_difference = 0
  for i in range(len(movie1)):
    length_difference += (movie1[i] - movie2[i]) ** 2
  return length_difference ** 0.5

## Part 4: 